In [67]:
import os

import numpy as np
import mne

from hdf5storage import loadmat, savemat 
from mne.minimum_norm import make_inverse_operator, apply_inverse, compute_source_psd_epochs, write_inverse_operator

from matplotlib import pyplot as plt


In [68]:
# for  ses in range(len(ses_num)):
#     ses_ID=ses_num[ses]
#     os.mkdir(str(ses_ID))

In [69]:
ses=0 # *********************************************************
ses_num=[20220713,20220721,20220804,20220808,20220810,20220811,20220815,20220816,20221003,2022100401,2022100402,20221005]
ses_ID=ses_num[ses]

outdict=loadmat('../../Cleaned_data/clean_'+str(ses_ID)+'.mat')
dataL=outdict['dataL']
dataR=outdict['dataR']
data=[outdict['dataL'],outdict['dataR']]

In [96]:
subj=0 # ************************************************************
tr=2  # ************************************************************
preprocessed_eeg=np.transpose(data[subj][0][tr][:,0:32]*0.000001) # reduce the amplitube to be shown on MNE's plot
sampling_freq=2000

In [97]:
# ch_bad=[] # ch_bad=outdict['ch_bad'][0]-1
# ch_dubious=[] # ch_dubious=outdict['ch_dubious'][0]-1
# n_channels=np.shape(preprocessed_eeg)[0]
# ch_info_bads=np.concatenate((ch_bad, ch_dubious), axis=0)
# ch_info_bads=np.unique(ch_info_bads)

In [98]:
# ch_labels=outdict['ch_labels'].tolist()
# ch_labels_names=list()
# for i in range(32):
#     ch_labels_names.append(ch_labels[i][0][0])

In [99]:
# ch_names = [f"E{n}" for n in range(1, 33)]
# ch_types = ["eeg"] * n_channels
# info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
# info.set_montage("GSN-HydroCel-32",match_case=False,match_alias=False,on_missing='raise', verbose=None)
# info["description"] = 'TMSi_32'
# print(info)


In [100]:
# outdict=loadmat('./xyzlabels.mat') # (this xyzlabels.mat has coordinates from original Poly to EDF to MNE)
# x=outdict['x']
# y=outdict['y']
# z=outdict['z']

# # this set of xyz does not look right to work as digitaization points for coregistartion, but works for topoplot
# for ch in range(3,len(info['dig'])):
#     info['dig'][ch]['r'][0]=x[ch-3]
#     info['dig'][ch]['r'][1]=y[ch-3]
#     info['dig'][ch]['r'][2]=z[ch-3]

# for ch in range(32):
#     info['chs'][ch]['loc'][0]=x[ch]
#     info['chs'][ch]['loc'][1]=y[ch]
#     info['chs'][ch]['loc'][2]=z[ch]

# Coordinates=np.zeros((32,3))
# Coordinates[:,0]=outdict['x']
# Coordinates[:,1]=outdict['y']
# Coordinates[:,2]=outdict['z']

In [101]:
# # mark the bad channels
# ch_labels_info_bads=list()
# for k in range(len(ch_info_bads)):
#     ch_labels_info_bads.append(ch_names[ch_info_bads[k]])

# info['bads'] = ch_labels_info_bads

In [102]:
raw = mne.io.RawArray(preprocessed_eeg, info)
raw.set_eeg_reference('average', projection=True)

Creating RawArray with float64 data, n_channels=32, n_times=251875
    Range : 0 ... 251874 =      0.000 ...   125.937 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


<RawArray | 32 x 251875 (125.9 s), ~61.5 MB, data loaded>

In [103]:
# raw.save("raw.fif",overwrite=True)

forward solution

In [104]:
# # the raw file containing the channel location + types
# # raw_fname = './raw.fif'

# # The paths to Freesurfer reconstructions
# subjects_dir ='../../archive/subjects/'
# subject = 'fsaverage' # change it to use freesurfer's bem

# # Compute Source Space (surface)
# ico = 4 #**************************************************************
# spacing='ico'+str(ico) 
# src = mne.setup_source_space(subject, spacing=spacing, add_dist='patch',
#                              subjects_dir=subjects_dir)
# conductivity = (0.3, 0.0075, 0.3)  #  three layers for EEG (MNE default  (0.3 0.006 0.3) )
# model = mne.make_bem_model(subject=subject, ico=ico,
#                            conductivity=conductivity,
#                            subjects_dir=subjects_dir)
# # 5 s


# bem = mne.make_bem_solution(model)
# # 1m 10s

# trans = './tmsi_trans.fif'


In [105]:
fwd = mne.make_forward_solution(raw.info, trans=trans, src=src, bem=bem,
                                meg=False, eeg=True, mindist=5.0, n_jobs=2,
                                verbose=True)
print(fwd)

# 10s


Source space          : <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=2562>, <surface (rh), n_vertices=163842, n_used=2562>] MRI (surface RAS) coords, subject 'fsaverage', ~30.8 MB>
MRI -> head transform : ./tmsi_trans.fif
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 5124 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999231 -0.035340  0.017012      -2.10 mm
     0.036517  0.996533 -0.074755      12.96 mm
    -0.014311  0.075319  0.997057      -2.61 mm
     0.000000  0.000000  0.000000       1.00

Read  32 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.



Employing the head->MRI coordinate transform with the BEM model.
BEM model instance of ConductorModel is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (will take a few...)
Checking surface interior status for 2562 points...
    Found  609/2562 points inside  an interior sphere of radius   47.7 mm
    Found    0/2562 points outside an exterior sphere of radius   98.3 mm
    Found    0/1953 points outside using surface Qhull


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


    Found    0/1953 points outside using solid angles
    Total 2562/2562 points inside the surface
Interior check completed in 456.9 ms
Checking surface interior status for 2562 points...
    Found  561/2562 points inside  an interior sphere of radius   47.7 mm
    Found    0/2562 points outside an exterior sphere of radius   98.3 mm
    Found    0/2001 points outside using surface Qhull


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


    Found    0/2001 points outside using solid angles
    Total 2562/2562 points inside the surface
Interior check completed in 500.1 ms

Setting up for EEG...
Computing EEG at 5124 source locations (free orientations)...


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.5s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.



Finished.
<Forward | MEG channels: 0 | EEG channels: 32 | Source space: Surface with 5124 vertices | Source orientation: Free>


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.7s finished


In [106]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
source_rr=fwd_fixed['source_rr']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...


    [done]
Leadfield size : 32 sensors x 5124 dipoles


In [107]:
# mne.write_forward_solution('fwd.fif', fwd_fixed, overwrite=True, verbose=None)

In [108]:
# save outputs all in one mat file. 
outdict=dict()
outdict['leadfield']=leadfield # leadfield matrix
savemat('leadfield.mat',outdict)
outdict=dict()
outdict['source_rr']=source_rr # the source location AND labels (matlab)
savemat('source_rr.mat',outdict)

# outdict['conductivity']=conductivity # 3 layrs conductivity

# outdict['ch_names']=ch_names #  
# outdict['Coordinates']=Coordinates #  

# outdict['ses_ID']=ses_ID #  
# outdict['subj_ind']=subj
# outdict['trl_ind']=tr

# savemat('./'+str(ses_ID)+'/'+'ses_'+str(ses_ID)+'_subj_'+str(subj)+'_trl_'+str(tr)+'.mat',outdict)

